<a href="https://colab.research.google.com/github/Rogerx98/ent-arch-big-data-project/blob/main/fine_tune_spaCy_NER_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Spacy NER Model 

In this Notebook we fine-tune a BERT Transformer with spaCy 3 to extract entities such as skills, degree and years of experience out of the full job description.
The train and test files (train.tsv, test.tsv) were downloaded from this github repository https://github.com/UBIAI/Fine_tune_BERT_with_spacy3. The training is based on the spacy 3 documentation: https://spacy.io/usage/training#basics

### Fine-tune spaCy model with train and test data

In [2]:
!nvidia-smi # check if GPU is running

Sat Jul 16 09:07:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive')
#set path to data in Google Drive
data_path = "/content/drive/MyDrive/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import spacy

In [12]:
#convert train and test files to json format 
!python -m spacy convert "/content/drive/MyDrive/Big Data/train.tsv" ./ -t json -n 1 -c iob
!python -m spacy convert "/content/drive/MyDrive/Big Data/test.tsv" ./ -t json -n 1 -c iob

ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (1 documents): train.json
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (1 documents): test.json


In [ ]:
#convert train and test json files to spacy readable format
!python -m spacy convert train.json ./ -t spacy
!python -m spacy convert test.json ./ -t spacy

In [15]:
# install spacy transformer pipeline and update spacy to a specific version (to work with the transformer)
!pip install spacy==3.3.0.dev0
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.2 MB 8.3 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: spacy
    Found existing installation: spacy 3.3.1
    Uninstalling spacy-3.3.1:
      Successfully uninstalled spacy-3.3.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 25 kB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 1.1 MB 12.2 MB/s 
     |████████████████████████████████| 4.4 MB 46.2 MB/s 
     |████████████████████████████████| 101 kB 10.0 MB/s 
     |████████████████████████████████| 596 kB 51.5 MB/s 
     |████████████████████████████████| 6.6 MB 33.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [ ]:
#install cuda-9.2 
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
# install pytorch transfomer library
pip install torch==1.7.1+cu92 torchvision==0.8.2+cu92 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
#Install spacy transformers for cuda 9.2 and change the CUDA_PATH and LD_LIBRARY_PATH
!export CUDA_PATH="/usr/local/cuda-9.2"
!pip install -U spacy[cuda92,transformers]

In [ ]:
#!pip install cupy-cuda100 (the only version that worked with the version of the transformer)
!pip install cupy-cuda100

In [ ]:
!export LD_LIBRARY_PATH=$CUDA_PATH/lib64:$LD_LIBRARY_PATH

In [ ]:
#import SpaCy 3 config file config.cfg which contains the model training components to train the model
#We need tp auto-fill the config file with the rest of the parameters for the BERT model
!python -m spacy init fill-config "/content/drive/MyDrive/Big Data/base_config.cfg" "/content/drive/MyDrive/Big Data/config_spacy.cfg"

In [ ]:
#depug the file and adjust according to the error
!python -m spacy debug data "/content/drive/MyDrive/Big Data/config_spacy.cfg"

In [ ]:
#if it does not run, downgrading spacy might help (I do not remeber what exact spacy version finally worked out)
#pip install -U spacy==3.1

In [ ]:
#train and test the model, -g 0 activates GPU, define the output folder
!python -m spacy train -g 0 "/content/drive/MyDrive/Big Data/config_spacy.cfg"  --output ./output --paths.train /content/train.spacy --paths.dev /content/test.spacy

In [ ]:
# zip the best model and download the file
#!zip -r /content/output/model-best.zip /content/output/model-best

### Test fine-tuned spacy model

In [16]:
#test model on an example
import spacy
!pip install spacy-transformers
#import spacy_transformers
nlp = spacy.load("en_core_web_sm")
#load pretrained model
nlp = spacy.load(data_path+"Big Data/content/output/model-best")
#load df with full text descriptions

In [29]:
#load df (containing the data from adzuna API and the screaped full text description of the job postings)
import pandas as pd
df = pd.read_csv(data_path+'Big Data/data copy.csv')

In [31]:
#REMOVE HTML TAGS FROM FULL_DESCRIPTION
import re
def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result
df['full_description']=df['full_description'].apply(lambda cw : remove_tags(cw))

In [35]:
df = df.head(5)
df

,Unnamed: 0,Unnamed: 0.1,title,contract_type,description,salary_max,salary_min,location.display_name,salary_is_predicted,redirect_url,created,full_description
0,1,Data Engineer - Data Integration,NaN,"Salary 80,000 - 110,000 USD per year Requireme...",Doximity,110000.0,80000,"Potrero Hill, San Francisco",0,https://www.adzuna.com/land/ad/2611763187?se=q...,2021-11-04T17:31:30Z,"Salary 80,000 - 110,000 USD per year Requireme..."
1,2,Data Scientist,NaN,"Salary 100,000 - 120,000 USD per year Requirem...",Cohesion,120000.0,100000,"West Loop, Chicago",0,https://www.adzuna.com/land/ad/2630782108?se=q...,2021-11-10T17:42:11Z,"Salary 100,000 - 120,000 USD per year Requirem..."
2,3,Data Analyst,NaN,"Salary 70,000 - 100,000 USD per year Requireme...",Doximity,100000.0,70000,"Potrero Hill, San Francisco",0,https://www.adzuna.com/land/ad/2611763219?se=q...,2021-11-04T17:31:30Z,"Salary 70,000 - 100,000 USD per year Requireme..."
3,4,Data Engineer,NaN,"Salary 110,000 - 140,000 USD per year Requirem...",Snap Inc.,140000.0,110000,"Loco, Lincoln County",0,https://www.adzuna.com/land/ad/2770687741?se=q...,2021-12-27T14:59:52Z,"Salary 110,000 - 140,000 USD per year Requirem..."
4,5,Data Scientist,NaN,"Salary 100,000 - 130,000 USD per year Requirem...",U.S. Gov,130000.0,100000,"Washington, Montgomery County",0,https://www.adzuna.com/land/ad/3174834461?se=q...,2022-05-27T17:56:16Z,"Salary 100,000 - 130,000 USD per year Requirem..."


In [36]:
#apply spacy pipeline to the first 5 rows of the column full_description
for doc in nlp.pipe(df.full_description, disable=["tagger", "parser"]):
    print([(ent.text, ent.label_) for ent in doc.ents])

[('data engineering', 'SKILLS'), ('operations', 'SKILLS'), ('data pipelines', 'SKILLS'), ('Python', 'SKILLS'), ('user-facing', 'SKILLS'), ('data warehousing', 'SKILLS'), ('Snowflake', 'SKILLS'), ('designing data modelsYou', 'SKILLS'), ('data structures', 'SKILLS'), ('Data engineers', 'SKILLS'), ('data integration', 'SKILLS'), ('user-facing', 'SKILLS'), ('.', 'SKILLS'), ('data products', 'SKILLS'), ('.', 'SKILLS'), ('data engineers', 'SKILLS'), ('data integration', 'SKILLS'), ('Python-', 'SKILLS'), ('SQL- MySQL-', 'SKILLS'), ('Big Data-', 'SKILLS'), ('Data-', 'SKILLS'), ('healthcare industry', 'SKILLS'), ('.', 'SKILLS'), ('.', 'SKILLS'), ('—', 'SKILLS'), ('.', 'SKILLS'), ('Healthcare is a universal concern', 'SKILLS'), ('.', 'SKILLS'), ('.', 'SKILLS')]
[('•', 'SKILLS'), ('scripting languages', 'SKILLS'), ('Python', 'SKILLS'), ('R', 'SKILLS'), ('data manipulation', 'SKILLS'), ('building data product prototypes•', 'SKILLS'), ('analytical', 'SKILLS'), ('Microsoft BI', 'SKILLS'), ('SQL', 'S

We successfully extracted the entites "SKILLS", "EXPERIENCE" and "DIPLOMA_MAJOR"

In [37]:
#for-loop to extract skills only
text = df.full_description
skills = []
for doc in nlp.pipe(text, disable=["tagger"]):
  for e in doc.ents:
    if e.label_ == "SKILLS":
      skills.append(e.text)
    else: continue 

In [38]:
#put skills in a df
df_skills = pd.DataFrame(skills, columns=["skills"])
df_skills

,skills
0,data engineering
1,operations
2,data pipelines
3,Python
4,user-facing
...,...
202,Performance Analysis and Integrity
203,business line activities
204,operational performance
205,.
